In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
# 身長,体重,ラベルのCSVデータを読み出す --- (※1)
csv = pd.read_csv("bmi.csv")

In [3]:
# データを正規化 --- (※2)
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100

In [4]:
# ラベルを三次元のクラスで表す --- (※3)
# - thin=(1,0,0) / normal=(0,1,0) / fat=(0,0,1)
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

In [5]:
# 正解率を求めるためにテストデータを準備 --- (※4)
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

In [6]:
# データフローグラフを構築する --- (※5)
# データを入れるプレースホルダを宣言
x  = tf.placeholder(tf.float32, [None, 2], name="x") 
y_ = tf.placeholder(tf.float32, [None, 3], name="y_") 

In [7]:
# 変数を宣言 --- (※6)
with tf.name_scope('interface') as scope:
    W = tf.Variable(tf.zeros([2, 3]), name="W"); # 重み
    b = tf.Variable(tf.zeros([3]), name="b"); # バイアス
    # ソフトマックス回帰を定義 --- (※7)
    with tf.name_scope('softmax') as scope:
        y = tf.nn.softmax(tf.matmul(x, W) + b)

In [8]:
# モデルを訓練する --- (※8)
with tf.name_scope('loss') as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

In [9]:
with tf.name_scope('training') as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(cross_entropy)

In [10]:
# 正解率を求める
with tf.name_scope('accuracy') as scope:
    predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

In [12]:
# セッションを開始
with tf.Session() as sess:
    tw = tf.summary.FileWriter("log_dir", graph=sess.graph)
    sess.run(tf.global_variables_initializer()) #変数を初期化
    # テストデータを用いて学習させる
    for step in range(3500):
        i = (step * 100) % 14000
        rows = csv[1 + i : 1 + i + 100]
        x_pat = rows[["weight","height"]]
        y_ans = list(rows["label_pat"])
        fd = {x: x_pat, y_: y_ans}
        sess.run(train, feed_dict=fd)
        if step % 500 == 0:
            cre = sess.run(cross_entropy, feed_dict=fd)
            acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
            print("step=", step, "cre=", cre, "acc=", acc)

step= 0 cre= 108.663 acc= 0.3242
step= 500 cre= 57.5887 acc= 0.8904
step= 1000 cre= 45.0209 acc= 0.898
step= 1500 cre= 41.6543 acc= 0.9566
step= 2000 cre= 34.664 acc= 0.943
step= 2500 cre= 34.287 acc= 0.9674
step= 3000 cre= 26.8808 acc= 0.9726


In [14]:
# 最終的な正解率を求める
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("正解率=", acc)

RuntimeError: Attempted to use a closed Session.